In [1]:
# default_exp data

# Data

> Functions for pre-processing data frames before feeding them into a decision tree etc.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export core
def export_from_data_to_core():
    "export to a different module"
    pass

# Data

> Functions for pre-processing data frames before feeding them into a decision tree etc.

In [6]:
#export
# from decision_tree.imports import *
from decision_tree.core import *

## Dataframe pre-processing functions

This is all "borrowed" from https://github.com/fastai/fastai/blob/master/old/fastai/structured.py

In [7]:
#export
def add_dateparts(df, col):
    """converts a column of df from a datetime64 to many columns containing
    the information from the date - inplace."""
    targ_pre = re.sub('[Dd]ate$', '', col.name)
    attrs = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
             'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 
             'Is_year_end', 'Is_year_start']
    for attr in attrs: df[targ_pre + attr] = getattr(col.dt, attr.lower())
    df[targ_pre + 'Elapsed'] = col.astype(np.int64) // 10**9
    df.drop(col.name, axis=1, inplace=True)

In [4]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 10_data.ipynb.
Converted index.ipynb.
